In [11]:
# 모듈 불러오기
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

In [12]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import *
import pandas as pd

titanic_df = pd.read_csv("../../data/titanic_train.csv")
titanic_df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [13]:

# 전처리
mean_data = titanic_df["Age"].mean()
titanic_df["Age"] = titanic_df["Age"].fillna(titanic_df["Age"].mean())
titanic_df["Cabin"] = titanic_df["Cabin"].fillna("N")
titanic_df["Embarked"] = titanic_df["Embarked"].fillna("N")

le_Sex = LabelEncoder()
le_Sex.fit(titanic_df["Sex"])
tmp = le_Sex.transform(titanic_df["Sex"])
titanic_df["Sex"] = tmp

le_Embarked = LabelEncoder()
le_Embarked.fit(titanic_df["Embarked"])
tmp = le_Embarked.transform(titanic_df["Embarked"])
titanic_df["Embarked"] = tmp

drop_feature = ["PassengerId", "Name", "Ticket", "Cabin"]
titanic_df = titanic_df.drop(drop_feature, axis=1)

y = titanic_df["Survived"]
X = titanic_df.drop(["Survived"], axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=11)

In [24]:
from sklearn.pipeline import make_pipeline

estimators = [
    ("dt", DecisionTreeClassifier()),
    ("rf", RandomForestClassifier()),
    ("knn", make_pipeline(StandardScaler(), KNeighborsClassifier())),  # 스케일링 후 knn 진행
    ("xgb", XGBClassifier()),
    ("lgb", LGBMClassifier())
]

model = VotingClassifier(estimators=estimators)

In [25]:
model.fit(X_train, y_train)

VotingClassifier(estimators=[('dt', DecisionTreeClassifier()),
                             ('rf', RandomForestClassifier()),
                             ('knn',
                              Pipeline(steps=[('standardscaler',
                                               StandardScaler()),
                                              ('kneighborsclassifier',
                                               KNeighborsClassifier())])),
                             ('xgb',
                              XGBClassifier(base_score=None, booster=None,
                                            callbacks=None,
                                            colsample_bylevel=None,
                                            colsample_bynode=None,
                                            colsample_bytree=None, device=None,
                                            ear...
                                            importance_type=None,
                                            interaction_constraints=None,
                                            learning_rate=None, max_bin=None,
                                            max_cat_threshold=None,
                                            max_cat_to_onehot=None,
                                            max_delta_step=None, max_depth=None,
                                            max_leaves=None,
                                            min_child_weight=None, missing=nan,
                                            monotone_constraints=None,
                                            multi_strategy=None,
                                            n_estimators=None, n_jobs=None,
                                            num_parallel_tree=None,
                                            random_state=None, ...)),
                             ('lgb', LGBMClassifier())])

In [26]:
from sklearn.metrics import *


y_pred = model.predict(X_test)

print("정확도:", accuracy_score(y_test, y_pred))
print("재현율:", recall_score(y_test, y_pred))
print("정밀도:", precision_score(y_test, y_pred))
print("f1점수:", f1_score(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))


정확도: 0.8715083798882681
재현율: 0.7704918032786885
정밀도: 0.8392857142857143
f1점수: 0.8034188034188035
[[109   9]
 [ 14  47]]
